<a href="https://colab.research.google.com/github/rselent/DS-Unit-2-Sprint-1-Linear-Models/blob/master/module3-ridge-regression/2_1_3_assignment_regression_classification_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 1, Module 3*

---

# Ridge Regression

## Assignment

We're going back to our other **New York City** real estate dataset. Instead of predicting apartment rents, you'll predict property sales prices.

But not just for condos in Tribeca...

- [ ] Use a subset of the data where `BUILDING_CLASS_CATEGORY` == `'01 ONE FAMILY DWELLINGS'` and the sale price was more than 100 thousand and less than 2 million.
- [ ] Do train/test split. Use data from January — March 2019 to train. Use data from April 2019 to test.
- [ ] Do one-hot encoding of categorical features.
- [ ] Do feature selection with `SelectKBest`.
- [ ] Fit a ridge regression model with multiple features. Use the `normalize=True` parameter (or do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html) beforehand — use the scaler's `fit_transform` method with the train set, and the scaler's `transform` method with the test set)
- [ ] Get mean absolute error for the test set.
- [ ] As always, commit your notebook to your fork of the GitHub repo.

The [NYC Department of Finance](https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page) has a glossary of property sales terms and NYC Building Class Code Descriptions. The data comes from the [NYC OpenData](https://data.cityofnewyork.us/browse?q=NYC%20calendar%20sales) portal.


## Stretch Goals

Don't worry, you aren't expected to do all these stretch goals! These are just ideas to consider and choose from.

- [ ] Add your own stretch goal(s) !
- [ ] Instead of `Ridge`, try `LinearRegression`. Depending on how many features you select, your errors will probably blow up! 💥
- [ ] Instead of `Ridge`, try [`RidgeCV`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html).
- [ ] Learn more about feature selection:
    - ["Permutation importance"](https://www.kaggle.com/dansbecker/permutation-importance)
    - [scikit-learn's User Guide for Feature Selection](https://scikit-learn.org/stable/modules/feature_selection.html)
    - [mlxtend](http://rasbt.github.io/mlxtend/) library
    - scikit-learn-contrib libraries: [boruta_py](https://github.com/scikit-learn-contrib/boruta_py) & [stability-selection](https://github.com/scikit-learn-contrib/stability-selection)
    - [_Feature Engineering and Selection_](http://www.feat.engineering/) by Kuhn & Johnson.
- [ ] Try [statsmodels](https://www.statsmodels.org/stable/index.html) if you’re interested in more inferential statistical approach to linear regression and feature selection, looking at p values and 95% confidence intervals for the coefficients.
- [ ] Read [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf), Chapters 1-3, for more math & theory, but in an accessible, readable way.
- [ ] Try [scikit-learn pipelines](https://scikit-learn.org/stable/modules/compose.html).

In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [0]:
import pandas as pd
import pandas_profiling

# Read New York City property sales data
df = pd.read_csv(DATA_PATH+'condos/NYC_Citywide_Rolling_Calendar_Sales.csv')

# Change column names: replace spaces with underscores
df.columns = [col.replace(' ', '_') for col in df]

# SALE_PRICE was read as strings.
# Remove symbols, convert to integer
df['SALE_PRICE'] = (
    df['SALE_PRICE']
    .str.replace('$','')
    .str.replace('-','')
    .str.replace(',','')
    .astype(int)
)

In [0]:
# BOROUGH is a numeric column, but arguably should be a categorical feature,
# so convert it from a number to a string
df['BOROUGH'] = df['BOROUGH'].astype(str)

In [0]:
# Reduce cardinality for NEIGHBORHOOD feature

# Get a list of the top 10 neighborhoods
top10 = df['NEIGHBORHOOD'].value_counts()[:10].index

# At locations where the neighborhood is NOT in the top 10, 
# replace the neighborhood with 'OTHER'
df.loc[~df['NEIGHBORHOOD'].isin(top10), 'NEIGHBORHOOD'] = 'OTHER'

In [41]:
## PYTHON isn't letting me do (100000 < df['SALE_PRICE'] < 2000000), so here we gooooo
df = df[( ((df['SALE_PRICE'] > 100000) & (df['SALE_PRICE'] < 2000000)) & 
           (df['BUILDING_CLASS_CATEGORY'] == '01 ONE FAMILY DWELLINGS') )]

pd.to_datetime( df.SALE_DATE)

train = df[ df[ 'SALE_DATE'] < '04/01/2019']
test = df[ df[ 'SALE_DATE'] > '03/31/2019']

df.shape, train.shape, test.shape

((3151, 21), (2507, 21), (644, 21))

In [90]:
###     5 STEPS:     ###    Linear Regression, SelectKBest = 0

## IMPORT
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.feature_selection import SelectKBest, f_regression
import category_encoders as ce

## INSTANTIATE
linReg = LinearRegression()

## ARRANGE target vector and features
target = 'SALE_PRICE'
highCardiB =[ 'ADDRESS','LAND_SQUARE_FEET','SALE_DATE',              # Cardi, B
              'BUILDING_CLASS_AT_TIME_OF_SALE', 'APARTMENT_NUMBER',  # Miss
              'EASE-MENT','APARTMENT_NUMBER'] # <-100% null values   # Cardi-nali-ty
features = train.columns.drop( [target] + highCardiB)

xTrain = train[ features]
yTrain = train[ target]
xTest = test[ features]
yTest = test[ target]

# ONE hot encoding
encoder = ce.OneHotEncoder( use_cat_names= True)
xTrain = encoder.fit_transform( xTrain)
xTest = encoder.transform( xTest)

## FIT model
linReg.fit( xTrain, yTrain)
yPred = linReg.predict( xTrain)
print( 'Train error: ${0:,.2f}'.format( mean_absolute_error( yTrain, yPred)) )

## APPLY model
yPred = linReg.predict( xTest)
print( 'Test error:  ${0:,.2f}'.format( mean_absolute_error( yTest, yPred)) )

Train error: $154,567.42
Test error:  $159,422.32


In [102]:
linReg.coef_

array([ 5.90349525e+14,  5.90349525e+14,  5.90349525e+14,  5.90349525e+14,
        5.90349525e+14, -7.98670081e+04,  4.13031271e+04, -4.09707812e+05,
        2.58322525e+05,  1.47796230e+05,  4.21586284e+04, -2.50000000e-01,
       -8.53675634e+04,  8.53681187e+04, -2.56786326e+01, -4.22650027e+01,
       -9.96707971e+04, -4.62192476e+04, -1.28033741e+05,  7.99006709e+03,
       -8.00846190e+03,  8.23727154e+04, -3.91291488e+05, -1.89623485e+04,
       -1.25865783e+05,  8.53676108e+04, -1.75956323e+05,  2.97912093e+05,
        5.20364743e+05, -1.43282869e+02, -8.53682233e+04,  2.04532672e+05,
        1.19164989e+05,  2.02140625e+02,  1.14658594e+03,  0.00000000e+00])

In [99]:
## SelectKBest = 1
print( 'Intercept\t\t', linReg.intercept_, '\n')

for k in range( 1, len( xTrain.columns) + 1):
    if k == 1:
        print( f'{k} feature')
    else:
        print( f'{k} features')

    selector = SelectKBest( score_func= f_regression, k= k)
    xTrainChosenOnes = selector.fit_transform( xTrain, yTrain)
    xTestChosenOnes = selector.transform( xTest)

    linReg.fit( xTrainChosenOnes, yTrain)
    yPred = linReg.predict( xTestChosenOnes)

    print( 'Test error:  ${0:,.2f}'.format( 
            mean_absolute_error( yTest, yPred)) )
    
print( pd.Series( linReg.coef_).to_string() )

Intercept		 -590349525073728.9 

1 feature
Test error:  $183,640.59
2 features
Test error:  $182,569.26
3 features
Test error:  $173,706.26
4 features
Test error:  $174,555.83
5 features
Test error:  $172,842.63
6 features
Test error:  $173,412.45
7 features
Test error:  $173,241.27
8 features
Test error:  $168,667.79
9 features
Test error:  $169,452.34
10 features
Test error:  $169,006.25
11 features
Test error:  $161,221.08
12 features
Test error:  $162,577.98
13 features
Test error:  $161,733.50
14 features
Test error:  $161,546.57
15 features


/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/univariate_selection.py:299: RuntimeWarning: divide by zero encountered in true_divide
  corr /= X_norms
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/univariate_selection.py:304: RuntimeWarning: invalid value encountered in true_divide
  F = corr ** 2 / (1 - corr ** 2) * degrees_of_freedom
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:1892: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/univariate_selection.py:299: RuntimeWarning: divide by zero encountered in

Test error:  $161,306.96
16 features
Test error:  $159,301.31
17 features
Test error:  $159,369.73
18 features
Test error:  $159,369.73
19 features
Test error:  $159,369.73
20 features
Test error:  $159,092.44
21 features
Test error:  $158,971.57
22 features
Test error:  $158,903.89
23 features
Test error:  $159,704.27
24 features
Test error:  $159,755.40
25 features


/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/univariate_selection.py:299: RuntimeWarning: divide by zero encountered in true_divide
  corr /= X_norms
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/univariate_selection.py:304: RuntimeWarning: invalid value encountered in true_divide
  F = corr ** 2 / (1 - corr ** 2) * degrees_of_freedom
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:1892: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/univariate_selection.py:299: RuntimeWarning: divide by zero encountered in

Test error:  $159,286.54
26 features
Test error:  $159,291.21
27 features
Test error:  $158,952.96
28 features
Test error:  $159,761.77
29 features
Test error:  $159,272.99
30 features
Test error:  $159,343.46
31 features
Test error:  $159,457.29
32 features
Test error:  $159,432.53
33 features
Test error:  $159,432.23
34 features
Test error:  $159,444.78
35 features


/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/univariate_selection.py:299: RuntimeWarning: divide by zero encountered in true_divide
  corr /= X_norms
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/univariate_selection.py:304: RuntimeWarning: invalid value encountered in true_divide
  F = corr ** 2 / (1 - corr ** 2) * degrees_of_freedom
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:1892: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/univariate_selection.py:299: RuntimeWarning: divide by zero encountered in

Test error:  $159,444.78
36 features
Test error:  $159,422.32


/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/univariate_selection.py:299: RuntimeWarning: divide by zero encountered in true_divide
  corr /= X_norms
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/univariate_selection.py:304: RuntimeWarning: invalid value encountered in true_divide
  F = corr ** 2 / (1 - corr ** 2) * degrees_of_freedom
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:1892: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/univariate_selection.py:299: RuntimeWarning: divide by zero encountered in

ValueError: ignored